In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm as tqdm
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("/Users/marzie/Documents/Data/followers_df.csv")
df = df.drop(['Unnamed: 0', 'op_time'], axis=1)

In [3]:
df

,podcaster_id,user_id
0,21543729,38719947
1,21543729,37434280
2,21543729,22013860
3,21543729,28885466
4,21543729,30848189
...,...,...
24854732,18385784,23356976
24854733,18385784,21762370
24854734,18385784,11046796
24854735,18385784,11486103


## Create user_podcast matrix

In [85]:
matrix_rows = df['podcaster_id'].unique()
len(matrix_rows)

3325

In [86]:
matrix_columns = df['user_id'].unique()
len(matrix_columns)

4316446

In [87]:
matrix = np.zeros((len(matrix_rows), len(matrix_columns)), dtype=bool)

for i in tqdm(range(len(matrix_rows))):
    
    mask = (df['podcaster_id'] == matrix_rows[i])
    
    index = np.where(np.in1d(matrix_columns, np.array(df[mask]['user_id'])))[0]
    
    matrix[i][index] = True

100%|███████████████████████████████████████| 3325/3325 [11:05<00:00,  5.00it/s]


In [144]:
%%time
number_of_followers = np.sum(matrix, axis=1)

CPU times: user 10.1 s, sys: 2.62 s, total: 12.7 s
Wall time: 15.7 s


In [145]:
#np.save("/Users/marzie/Documents/Data/number_of_followers", number_of_followers)

In [147]:
minimum_follower = 50
maximum_follower = 20000

In [148]:
mask1 = number_of_followers < maximum_follower

mask2 = number_of_followers > minimum_follower

mask = mask1*mask2

In [150]:
#plt.hist(number_of_followers[mask])

In [158]:
len(number_of_followers[mask])

1878

In [159]:
matrix_updated = matrix[mask]

matrix_rows_updated = matrix_rows[mask]

number_of_followers_updated = number_of_followers[mask]

In [160]:
number_of_podcasters = len(matrix_rows_updated)

common_followers = np.zeros((number_of_podcasters, number_of_podcasters))

for i in tqdm(range(number_of_podcasters)):
    for j in range(i+1, number_of_podcasters):
        
        common_followers[i, j] = (matrix_updated[i]*matrix_updated[j]).sum()

100%|█████████████████████████████████████| 1878/1878 [1:43:24<00:00,  3.30s/it]


In [197]:
n = len(common_followers)

i_lower = np.tril_indices(n, -1)
common_followers[i_lower] = common_followers.T[i_lower]  

np.fill_diagonal(common_followers, number_of_followers_updated)

In [185]:
#np.save("/Users/marzie/Documents/Data/common_followers", common_followers)

In [188]:
#np.save("/Users/marzie/Documents/Data/podcaster_ids_array", matrix_rows_updated)

In [211]:
common_followers

array([[207.,   1.,   0., ...,   1.,   1.,   0.],
       [  1., 496.,   0., ...,   0.,   0.,   1.],
       [  0.,   0., 110., ...,   0.,   0.,   0.],
       ...,
       [  1.,   0.,   0., ...,  70.,   0.,   0.],
       [  1.,   0.,   0., ...,   0.,  58.,   0.],
       [  0.,   1.,   0., ...,   0.,   0., 110.]])